In [24]:
import geopandas as gpd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [25]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [26]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [27]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)
map_before = geemap.Map(center=[-11.5, -63.25], zoom=8)
map_after = geemap.Map(center=[-11.5, -63.25], zoom=8)

In [28]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-01-01', '2016-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer_diff)
satelite_2024 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2024-01-01', '2024-06-30').filterBounds(pacaas_area).map(mask_clouds).median().clip(buffer_diff)

In [29]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [30]:
before_ndvi = calc_ndvi(satelite_2016).select('NDVI')
after_ndvi = calc_ndvi(satelite_2024).select('NDVI')

change = after_ndvi.subtract(before_ndvi).rename('NDVI CHANGE')